# Prédiction de données

In [1]:
# Importer les bibliothèques
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# Charger le fichier CSV
df = pd.read_csv('data/houses_Madrid_cleaned.csv')

In [3]:
# Définir la taille de l'échantillon, par exemple 10% du dataset
sample_size = 0.01  # 1% du dataset

# Créer un échantillon aléatoire du dataset
df_sample = df.sample(frac=sample_size)

# Définir les colonnes cibles et les colonnes à supprimer
target_column = 'buy_price'
drop_columns = ['title', 'subtitle', 'raw_address', 'street_name']  # Colonnes non pertinentes (strings d'adresses)

# Créer X et y à partir de l'échantillon
X_sample = df_sample.drop(drop_columns + [target_column], axis=1)
y_sample = df_sample[target_column]

# Séparer les colonnes numériques et catégorielles dans l'échantillon
numerical_columns_sample = X_sample.select_dtypes(include=['float64', 'int64']).columns
categorical_columns_sample = X_sample.select_dtypes(include=['object']).columns

print("Colonnes numériques dans l'échantillon :", numerical_columns_sample)
print("Colonnes catégorielles dans l'échantillon :", categorical_columns_sample)

Colonnes numériques dans l'échantillon : Index(['id', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'rent_price',
       'buy_price_by_area'],
      dtype='object')
Colonnes catégorielles dans l'échantillon : Index(['floor', 'is_floor_under', 'neighborhood_id', 'operation',
       'house_type_id', 'is_new_development', 'has_central_heating',
       'has_individual_heating', 'has_lift', 'is_exterior',
       'energy_certificate', 'is_orientation_north', 'is_orientation_west',
       'is_orientation_south', 'is_orientation_east'],
      dtype='object')


In [4]:


# Créer X et y
X = df.drop(drop_columns + [target_column], axis=1)
y = df[target_column]

# Séparer les colonnes numériques et catégorielles
numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

print("Colonnes numériques :", numerical_columns)
print("Colonnes catégorielles :", categorical_columns)


Colonnes numériques : Index(['id', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'rent_price',
       'buy_price_by_area'],
      dtype='object')
Colonnes catégorielles : Index(['floor', 'is_floor_under', 'neighborhood_id', 'operation',
       'house_type_id', 'is_new_development', 'has_central_heating',
       'has_individual_heating', 'has_lift', 'is_exterior',
       'energy_certificate', 'is_orientation_north', 'is_orientation_west',
       'is_orientation_south', 'is_orientation_east'],
      dtype='object')


In [5]:
# Créer un preprocessor qui standardise les colonnes numériques et encode les colonnes catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ]
)

# Appliquer le preprocessor et séparer les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)


In [6]:
# Créer un pipeline avec le preprocessor et le modèle SVM
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])

# Entraîner le modèle
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['id', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'rent_price',
       'buy_price_by_area'],
      dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['floor', 'is_floor_under', 'neighborhood_id', 'operation',
       'house_type_id', 'is_new_development', 'has_central_heating',
       'has_individual_heating', 'has_lift', 'is_exterior',
       'energy_certificate', 'is_orientation_north', 'is_orientation_west',
       'is_orientation_south', 'is_orientation_east'],
      dtype='object'))])),
                ('classifier', SVC())])

In [7]:
# Définir la grille d'hyperparamètres
param_grid = {
    'classifier__C': [0.1, 1, 10, 100],
    'classifier__gamma': [1, 0.1, 0.01, 0.001],
    'classifier__kernel': ['linear', 'rbf']
}

In [8]:
# Créer l'objet GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)

# 4. Entraîner le modèle avec GridSearchCV
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


c:\Users\QWERTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\QWERTY\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['id', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'rent_price',
       'buy_price_by_area'],
      dtype='object')),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         Index(['floor', 'is_floor_under', 'neighborhood_id', 'operation',
       'house_type_id', 'is_new_development', 'has_centra...ng',
       'has_individual_heating', 'has_lift', 'is_exterior',
       'energy_certificate', 'is_orientation_north', 'is_orientation_west',
       'is_orientation_south', 'is_orientation_east'],
      dtype='object'))])),
                                       ('classifier', SVC())]),
             n_jobs=-1,
             param_grid={'classifier__C': [0.1, 1, 10, 100],
                         'classifier__gamma': [1, 0.1, 0.01, 0.001],
                         'classifier__kernel': ['linear', 'rbf']},
             verbose=2)

In [9]:
# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

# Prédire les résultats avec le meilleur modèle trouvé
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

# Afficher la matrice de confusion et l'accuracy
print("Matrice de confusion :")
print(confusion_matrix(y_test, predictions))

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

Meilleurs hyperparamètres trouvés :
{'classifier__C': 0.1, 'classifier__gamma': 1, 'classifier__kernel': 'linear'}


ValueError: Found unknown categories ['Neighborhood 65: Cuatro Vientos (None €/m2) - District 10: Latina'] in column 2 during transform